In [7]:
import re
import sqlite3
import pandas as pd
import numpy as np
import gzip
from datetime import datetime
from IPython.display import clear_output
from xml.etree import ElementTree as et

dbname = 'discogs-reddit-fin2.db'
 

In [2]:
def run_sql(sql):
    """
    Excutes a valid SQL Statement.
    Parameters:
    sql: sql string value
    Returns: None
    """
    print(sql)
    conn = sqlite3.connect(dbname)
    conn.execute(sql)
    conn.commit()
    conn.close()

In [3]:
"""
SQL Statements for creatiton of tables for the releases database.
"""
sql = """
    CREATE TABLE IF NOT EXISTS companies (
        release_id INT,
        company_id INT,
        name TEXT,
        catno TEXT,
        entity_type TEXT,
        entity_type_name TEXT,
        resource_url TEXT
    )
"""
run_sql(sql)


sql = """
    CREATE TABLE IF NOT EXISTS tracks (
        release_id INT,
        position INT,
        title TEXT,
        duration TEXT
    )
"""
run_sql(sql)

sql = """
    CREATE TABLE IF NOT EXISTS releases (
        release_id INT,
        status TEXT,
        title TEXT,
        country TEXT,
        released TEXT,
        genres TEXT,
        styles TEXT
    )
"""
run_sql(sql)

sql = """
    CREATE TABLE IF NOT EXISTS labels (
        release_id INT,
        labels_id INT,
        name TEXT,
        catno TEXT
    )
"""
run_sql(sql)

sql = """
    CREATE TABLE IF NOT EXISTS formats (
        release_id INT,
        qty INT,
        name TEXT,
        text TEXT,
        desc text
    )
"""
run_sql(sql)


sql = """
    CREATE TABLE IF NOT EXISTS artists (
        release_id INT,
        artist_id INT,
        name TEXT,
        anv TEXT,
        join_ TEXT,
        role TEXT,
        flag INT, 
        position INT
    )
"""
run_sql(sql)

sql = """
    CREATE TABLE IF NOT EXISTS releases_reddit (
        release_id INT,
        title TEXT,
        released TEXT,
        reddit_id TEXT,
        subject TEXT,
        body TEXT,
        created_utc TEXT
    )
"""
run_sql(sql)

sql = """
    CREATE TABLE IF NOT EXISTS artists_reddit (
        release_id INT,
        artists_id TEXT,
        name TEXT,
        released TEXT,
        reddit_id TEXT,
        subject TEXT,
        body TEXT,
        created_utc TEXT
    )
"""
run_sql(sql)


    CREATE TABLE IF NOT EXISTS companies (
        release_id INT,
        company_id INT,
        name TEXT,
        catno TEXT,
        entity_type TEXT,
        entity_type_name TEXT,
        resource_url TEXT
    )


    CREATE TABLE IF NOT EXISTS tracks (
        release_id INT,
        position INT,
        title TEXT,
        duration TEXT
    )


    CREATE TABLE IF NOT EXISTS releases (
        release_id INT,
        status TEXT,
        title TEXT,
        country TEXT,
        released TEXT,
        genres TEXT,
        styles TEXT
    )


    CREATE TABLE IF NOT EXISTS labels (
        release_id INT,
        labels_id INT,
        name TEXT,
        catno TEXT
    )


    CREATE TABLE IF NOT EXISTS formats (
        release_id INT,
        qty INT,
        name TEXT,
        text TEXT,
        desc text
    )


    CREATE TABLE IF NOT EXISTS artists (
        release_id INT,
        artist_id INT,
        name TEXT,
        anv TEXT,
        join_ TEXT,
        role TEXT

In [11]:
#do not run 
# tables = ['artists','formats','labels','releases','tracks','companies']
# for t in tables:
#     sql = "DELETE FROM " + t
#     #sql = "DELETE FROM " + t + " WHERE RELEASE_ID = '21731164'"
#     run_sql(sql)
# print('records deleted')

#do not run 
tables = ['artists','formats','labels','releases','tracks','companies']

#sql = "DROP TABLE releases_reddit"
#run_sql(sql)
sql = "DROP TABLE artists_reddit"
run_sql(sql)
print('records deleted')

DROP TABLE artists_reddit
records deleted


In [5]:
def get_attrib(xml, attrib):
    """
    Searches for an attrobute in the xml string and returns value.
    
    Parameters:
    xml: string xml format
    attrib: string attributte name in xml 
    Returns: string value of attibute
    """
    
    txt = ''
    if xml.get(attrib) is not None:
        txt = xml.get(attrib)
    return txt

def get_text(xml, elem):
    """
    Searches for xpath in the xml string and returns text value.
    
    Parameters:
    xml: string xml format
    element: string element name
    Returns: string text of element
    """
    txt = ''
    if xml.find(elem) is not None:
        if xml.find(elem).text is not None:
            txt = xml.find(elem).text
    return txt

def get_text_append(xml, xpath):
    """
    Searches for xpath in the xml string and returns all text values.
    appended using '|'
    
    Parameters:
    xml: string xml format
    xpath: string xpath search pattern
    Returns: string of xpath values
    """
    txt = ''
    for t in xml.findall(xpath):
            if t.text is not None:
                txt += t.text + '|'        
    return txt[:-1]

def insert_sql(table, cols, data, conn):
    """
    Runs an Insert SQL Statement and uses the parameter values.
    
    Parameters:
    table: string table name
    conn: list of columns for the table
    data: data to be inserted to the table
    conn: SQLIte3 connection string
    
    
    Returns: None
    """
    sql = (""" INSERT INTO """ + table + """ (""" 
           + ','.join(x for x in cols) +
           """) VALUES (""" + ','.join('?' for x in cols) + """)""")
    conn.execute(sql, data)
    conn.commit()  

def save_artist_list(xml, xpath, conn, rel_id, flag=0, position=0):
    """
    Process an XML string of artists record.
    
    Parameters:
    xml: xml formatted string for music releases
    conn: sqllite connection
    rel_id: integer release id
    flag: integer 0=main artist, 1=extra artist, 2=track artists
    position: integer position in track within the release
    
    Returns: None
    """
    artist_cols = ['id', 'name', 'join', 'anv', 'role']
    
    for t in xml.findall(xpath):
        ret, cols = [], []
        for i in artist_cols:
            if t.find(i) is not None:
                if t.find(i).text is not None:
                    ret.append(t.find(i).text)
                    if i == 'join':
                        cols.append('join_')
                    elif i == 'id':
                        cols.append('artist_id')
                    else:
                        cols.append(i)

        ret.insert(0, rel_id)
        cols.insert(0, 'release_id')  
        ret.append(flag)
        ret.append(position)
        cols.append('flag')
        cols.append('position')
        
        insert_sql('artists', cols, ret, conn)
            

In [6]:
def process_release(xml, conn):
    """
    Process an XML string of release and saves each record including its 
    sub elements into SQLLite Tables.
    
    Parameters:
    xml: xml formatted string for music releases
    conn: sqllite connection
    
    Returns:None
    """
    data = []
    data.append(get_attrib(xml, 'id'))
    data.append(get_attrib(xml, 'status'))
    data.append(get_text(xml, 'title'))
    data.append(get_text(xml, 'country'))
    data.append(get_text(xml, 'released'))
    data.append(get_text_append(xml, './/genres/genre'))
    data.append(get_text_append(xml, './/styles/style'))

    conn.execute("""INSERT INTO releases VALUES (?, ?, ?, ?, ?, ?, ?)""", data)
    conn.commit()

    label_cols = ['id', 'name', 'catno']
    for t in xml.findall('.//labels/label'):
        ret, cols = [], []
        for i in label_cols:
            if t.get(i) is not None:
                ret.append(t.get(i))
                if i == 'id': 
                    cols.append('labels_id')
                else:
                    cols.append(i)

        ret.insert(0, data[0])
        cols.insert(0, 'release_id')
        insert_sql('labels', cols, ret, conn)


    format_cols = ['qty', 'name', 'text']
    for t in xml.findall('.//formats/format'):
        ret, cols = [], []
        for i in format_cols:
            if t.get(i) is not None:
                ret.append(t.get(i))
                cols.append(i)
        ret.append(get_text_append(xml, 
                                   './/formats/format/'
                                   'descriptions/description'))

        ret.insert(0, data[0])
        cols.insert(0, 'release_id')
        cols.append('desc')
        insert_sql('formats', cols, ret, conn)

    save_artist_list(xml, './/artists/artist', conn, data[0])
    save_artist_list(xml, './/extraartists/artist', conn, data[0], flag=1)

    track_cols = ['position', 'title', 'duration']
    for t in xml.findall('.//tracklist/track'):
        ret, cols = [], []
        pos, title = -1, ''
        for i in track_cols:
            if t.find(i) is not None:
                if t.find(i).text is not None:
                    ret.append(t.find(i).text)
                    cols.append(i)
                    if i == 'position':
                        pos = t.find(i).text
                    elif i == 'title':
                        title = t.find(i).text 
                    
        ret.insert(0, data[0])
        cols.insert(0, 'release_id')
        if pos == -1: 
            ret.insert(1, pos)
            cols.insert(1, 'position')
                
        if title != '':
            insert_sql('tracks', cols, ret, conn)
            save_artist_list(xml, './/extraartists/artist', conn, data[0],
                             flag=2, position=pos)

    comp_cols = (['id', 'name', 'catno', 'entity_type', 
                  'entity_type_name', 'resource_url'])

    for t in xml.findall('.//companies/company'):
        ret, cols = [], []
        for i in comp_cols:
            if t.find(i) is not None:
                if t.find(i).text is not None:
                    ret.append(t.find(i).text)
                    if i == 'id': 
                        cols.append('company_id')
                    else:
                        cols.append(i)
        ret.insert(0, data[0])
        cols.insert(0, 'release_id')
        insert_sql('companies', cols, ret, conn) 


In [18]:
"""
Process releases gz file and picks 2022 year writes into an XML to be
post procesed in a separate function.
"""
print('start')
path = "discogs_2022_xml.xml"
conn = sqlite3.connect(dbname)

with open(path, 'r') as fin:
    f = fin.readlines()
    re_run = False
    for line in f: 
       
        if line.startswith('<release '):
            str_xml = ''
            
        str_xml = str_xml + line

        if (line.endswith('</release>\n') and 
            str_xml.startswith('<release ')):
            xml = et.fromstring(str_xml)
     
            if xml.find('released') is not None:
                if xml.find('released').text is not None:
                        id_ = get_attrib(xml, 'id')
                        tm = str(datetime.now())
                        if int(id_) > 0 or re_run == False:
                            re_run = True
                            with open('lastload_xml' + tm + '.txt', 'a') as f:
                                f.writelines(str(fin.tell()) + ',' 
                                             + id_ + 
                                             ',' + tm 
                                             + '\n')
                            process_release(xml, conn)
print('fin')
conn.close()

start
fin


In [ ]:
"""
Process releases gz file and picks 2022 year writes into an XML to be
post procesed in a separate function.
"""
print('start')
path = "/mnt/data/public/discogs/2022/discogs_20221001_releases.xml.gz"


with gzip.open(path,'rb') as fin:
    str_xml = ''
    #start where 2022 data was found in the gz file
    fin.seek(4154910000)
    for line in fin: 
        decode = line.decode("utf-8")
        if decode.startswith('<release '):
            str_xml = ''
        str_xml = str_xml + decode
        
        if (decode.endswith('</release>\n') and 
            str_xml.startswith('<release ')):
            xml = et.fromstring(str_xml)
      
            if xml.find('released') is not None:
                if xml.find('released').text is not None:
                    if re.search(r'.*2022.*', xml.find('released').text):
                        id_ = get_attrib(xml, 'id')
                        tm = str(datetime.now()) 
                        with open('lastload' + tm + '.txt', 'a') as f:
                            f.writelines(str(fin.tell()) + ',' 
                                         + get_attrib(xml, 'id') + 
                                         ',' + tm
                                         + '\n')
                            print(fin.tell(), end='\r')
                        with open('discogs_2022_xml.xml', 'a') as f:
                            f.writelines(str_xml + '\n')
            
print('fin')


In [ ]:
"""
Load release and reddit submissions data
"""
conn = sqlite3.connect(dbname)
sql = """
      SELECT DISTINCT release_id, title, released
      FROM releases 
      WHERE length(trim(title)) >= 4 
      """
           
df_rel = pd.read_sql_query(sql, conn)
display(df_rel)

sql = """
      SELECT DISTINCT id as reddit_id, title as subject, selftext as body, created_utc
      FROM reddit_submissions_processed r
      """
df_subj = pd.read_sql_query(sql, conn)
display(df_subj)

conn.close()

In [ ]:
"""
Loads reddit_submissions witth matching title in the subject into 
reddit_staging table
"""

conn = sqlite3.connect(dbname)
str = ""
id_list = []
for i, row in df_rel.iterrows():
    str = "|".join([str,r'.*' + re.escape(row.title) + '.*'])
    id_list.append(row.release_id)
    if i % 200 == 0 or (len(df_rel)-1) == i:
        clear_output(wait=True)
        df_out = df_subj[df_subj.subject.str.match(str[1:])]
        df_subj = df_subj.drop(df_out.index)
        print(f'Row Number: {i}')
        print(f'df_out len: {len(df_out)}')
        df_out.to_sql('reddit_staging', con=conn, 
                          if_exists='append', index=False)
        str = ""      
conn.close()

In [ ]:
"""
Load reddit staging for post processing into the final table 
releases_reddit_fin
"""
conn = sqlite3.connect(dbname)
sql = """
      SELECT *
      FROM reddit_staging
      
      """
df_subj = pd.read_sql_query(sql, conn)
for i, row in df_rel[20000:40000].iterrows():

    regex = r'\b' + re.escape(row.title) + r'\b'
    df_out = df_subj[df_subj.subject.str.contains(regex)]
    if len(df_out) > 0:
        df_out = df_out.assign(release_id = row.release_id,
                               title = row.title,
                               released = row.released)
        df_out.to_sql('releases_reddit_fin', con=conn, 
                      if_exists='append', index=False)
    else:
        x = pd.DataFrame([row.release_id], columns=['release_id'])
        x.to_sql('exclude_id_fin_', con=conn, 
                          if_exists='append', index=False)
    clear_output(wait=True)
    print(f'Row Number: {i}')
    print(df_out)

display(df_subj)
conn.close()